In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [62]:
import pandas as pd

df1 = pd.read_csv('../dataset/distance_dataset.csv', index_col=False)
df2 = pd.read_csv('TON_IoT.csv', index_col=False).sample(2000, random_state=42)
df2 = df2.reset_index(drop=True)


df = pd.concat([df1, df2], axis=1)  # Set ignore_index=True

df.to_csv('distance_security_dastset.csv',index=False)
print(df)

      TaskFileSize  TaskOutputFileSize  TaskFileLength  UserLatitude  \
0               73                  20              24     54.183368   
1               33                  19              19    -30.022945   
2               98                  10              12    -31.519456   
3               68                  20              20    -58.924872   
4               76                  13              21    -39.961638   
...            ...                 ...             ...           ...   
1995            15                  12              11     52.943011   
1996            13                  14              11      0.473082   
1997           100                  10              24    -43.530012   
1998            42                  14              19    -40.006917   
1999            23                  15              23     65.620008   

      UserLongitude  DataCenterID          ts         src_ip  src_port  \
0         34.691517             5  1556022080   192.168.1.30 

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA


In [4]:
aes_key = get_random_bytes(32)  # AES-256
base64_encoded_aes_key = b64encode(aes_key).decode('utf-8')

'pmxj4w6SBu8BzW94kG36vdrMF6t8ksyt6h2HoiOsJp4='

In [5]:
input_file_path = "TON_IoT.csv"
encrypted_file_path = "encrypted_TON_IoT.csv"
decrypted_file_path = "decrypted_TON_IoT.csv"
def encrypt_file(input_file_path, output_file_path, key):
    # Initialize AES cipher with CBC mode
    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    # Read and encrypt the file
    with open(input_file_path, 'rb') as file:
        plaintext = file.read()
        ciphertext = cipher.encrypt(pad(plaintext, AES.block_size))
    # Write the IV followed by the ciphertext to the output file
    with open(output_file_path, 'wb') as file:
        file.write(iv + ciphertext)

encrypt_file(input_file_path, encrypted_file_path, aes_key)

In [6]:
rsa_key = RSA.generate(2048)
private_key = rsa_key.export_key()
public_key = rsa_key.publickey().export_key()
private_key_path = "private_key.pem"
public_key_path = "public_key.pem"
with open(private_key_path, 'wb') as file:
    file.write(private_key)

with open(public_key_path, 'wb') as file:
    file.write(public_key)

In [7]:
rsa_public_key = RSA.import_key(open(public_key_path).read())
cipher_rsa = PKCS1_OAEP.new(rsa_public_key)
encrypted_aes_key = cipher_rsa.encrypt(aes_key)
base64_encoded_encrypted_aes_key = b64encode(encrypted_aes_key).decode('utf-8')

In [8]:
rsa_private_key = RSA.import_key(open(private_key_path).read())
cipher_rsa_dec = PKCS1_OAEP.new(rsa_private_key)
decrypted_aes_key = cipher_rsa_dec.decrypt(encrypted_aes_key)
base64_encoded_decrypted_aes_key = b64encode(decrypted_aes_key).decode('utf-8')

In [9]:
def decrypt_file(input_file_path, output_file_path, key):
    with open(input_file_path, 'rb') as file:
        iv = file.read(16)
        ciphertext = file.read()
        
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    with open(output_file_path, 'wb') as file:
        file.write(plaintext)

decrypt_file(encrypted_file_path, decrypted_file_path, decrypted_aes_key)

In [10]:
key_match = decrypted_aes_key == aes_key
print('the decrypted AES key matches the original key') if key_match else 'the decrypted AES key does not matche the original key'

the decrypted AES key matches the original key


In [11]:
original_df = pd.read_csv(input_file_path)
decrypted_df = pd.read_csv(decrypted_file_path)
original_shape = original_df.shape
decrypted_shape = decrypted_df.shape

In [12]:
matching_rows = (original_df == decrypted_df).all(axis=1).sum()
percentage_matching = (matching_rows / len(original_df)) * 100
print('matching rows:')
print(matching_rows , '/', len(original_df))
print(percentage_matching, '% match')

matching rows:
461043 / 461043
100.0 % match


In [13]:
X= decrypted_df.drop(['label', 'type'], axis = 1)
y= decrypted_df['label']

In [14]:
le = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = le.fit_transform(X[col].astype(str))

In [15]:
scaler = MinMaxScaler()
scaledX = scaler.fit_transform(X)

In [16]:
num_nans = X.isna().sum().sum()
print(f'Total NaN values in df: {num_nans}')

Total NaN values in df: 0


In [17]:
X_train, X_test, y_train, y_test = train_test_split(scaledX, y, test_size=0.2, random_state=20)

In [18]:
def evaluate_classifier(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Plot the confusion matrix
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.colorbar()
    tick_marks = np.arange(len(set(y_test)))
    plt.xticks(tick_marks, sorted(set(y_test)))
    plt.yticks(tick_marks, sorted(set(y_test)))
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    # Add the numbers to the confusion matrix plot
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j], horizontalalignment='center', color='white' if cm[i, j] > cm.max() / 2. else 'black')

    plt.show()

    # Calculate the classification report
    report = classification_report(y_test, y_pred)
    print("Classification Report:")
    print(report)

    # Calculate sensitivity and specificity
    sensitivity = []
    specificity = []
    for i in range(cm.shape[0]):
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        tn = cm.sum() - tp - fp - fn
        sensitivity.append(tp / (tp + fn))
        specificity.append(tn / (tn + fp))
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)
